In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from IPython.display import display, HTML

In [ ]:
#Importer la data (attention modifier le nom du fichier csv en conséquence): 
df = pd.read_csv("2022.csv")
df.head()


# <font color='red'>1/ Informations générales :</font>

### <font color='blue'> Taille du dataframe : </font>

In [ ]:
dimensions = df.shape
dimensions

### <font color='blue'> Informations sur les variables : </font>

In [ ]:
# Définir une fonction pour afficher les informations en deux colonnes
def display_info_two_columns(df):
    # Créer un DataFrame temporaire pour stocker les informations
    info_df = df.dtypes.to_frame(name='type des variables')
    info_df['pourcentage des nan'] = df.isnull().sum()/ len(df) * 100
    
    # Diviser le DataFrame en deux parties
    midpoint = len(info_df) // 2
    info_df_left = info_df.iloc[:midpoint]
    info_df_right = info_df.iloc[midpoint:]
    
    # Afficher les deux parties côte à côte
    html_output = (
        '<div style="display: flex; justify-content: space-between;">'
        f'<div>{info_df_left.to_html()}</div>'
        f'<div>{info_df_right.to_html()}</div>'
        '</div>'
    )
        # Enregistrement du contenu HTML dans un fichier HTML
    with open('info_table.html', 'w') as file:
        file.write(html_output)

    # Affichage d'un lien pour télécharger le fichier
    display(HTML('<a href="info_table.html" download="info_table.html">Télécharger le tableau</a>'))

    
    # Afficher le résultat avec la fonction display
    display(HTML(html_output))

# Afficher les informations du DataFrame en deux colonnes
display_info_two_columns(df)

# <font color='red'>2/ Preprocessing :</font>

### <font color='blue'> Gérer les doublons : </font>

In [ ]:
num_duplicates = df.duplicated().sum()
num_duplicates
# Il n'a pas de doublons

### <font color='blue'> Graphiques avant Data Cleaning :  </font>

In [ ]:
df.boxplot(column = "Ewltp (g/km)", by= "Ft", figsize = (12,10))
df.boxplot(column = "Fuel consumption ", by= "Ft", figsize = (12,10))


### <font color='blue'> Analyse comparative des variables Fuel consumption et émissions de CO2: </font>

In [ ]:
# Temps d'execution long 
sns.pairplot(df[(df["Ft"] != 'electric') & (df["Ft"] != 'hydrogen')], 
             vars=['Fuel consumption ','Ewltp (g/km)'], hue= "Ft")

plt.title("Analyse comparative des variables Fuel consumption et émissions de CO2 (Ewltp (g/km)) pour tous les véhicules hors 100% électriques et à hydrogène")
plt.show();

In [ ]:
# Temps d'execution long 
sns.pairplot(df[(df['Ft'] == 'electric') | (df['Ft'] == 'hydrogen') | (df['Ft'] == 'petrol/electric') | (df['Ft'] == 'diesel/electric')], 
             vars=['m (kg)', 'W (mm)', 'At1 (mm)','z (Wh/km)', 'Electric range (km)'], 
             hue=''Ft')
plt.title("Analyse comparative des variables masse, empattement,voie avant, puissance moteur et autonomie pour tous  les véhicules hors ICE(petrol, diesel, e85, ng, lpg)")
plt.show();


### <font color='blue'> Supprimer les variables inutiles : </font>

In [ ]:
columns_to_drop = ['ID', 'VFN', 'Mh', 'Man', 'Tan','T', 'Va', 'Ve', 'Mk', 'Cn',
                  'Ct', 'Cr', 'r', 'Date of registration','Status', 'year', 'MMS', 'De', 'Vf']

df.drop(columns=columns_to_drop, axis = 1, inplace=True)
# On supprime les colonnes redondantes ou inutilisables (Mt,IT,Fm)
df = df.drop(["Mt","IT","Fm", "Fuel consumption "], axis = 1)

### <font color='blue'> Analyse préliminaire avant le nettoyage des données: </font>

In [ ]:
sns.kdeplot(data=df, x='Ewltp (g/km)',hue='Ft')

# Ajout du titre et des noms des axes
plt.title("Analyse de la distribution des émissions de CO2 par types de véhicules")
plt.xlabel("Emission_co2_(g/km)")

# Affichage du graphique
plt.show()

### <font color='blue'> Supprimer les nan de la variable cible: </font>

In [ ]:
df = df.dropna(axis = 0, subset="Ewltp (g/km)")

### <font color='blue'> Supprimer les variables avec plus de 80% de nan :  </font>

In [ ]:
# Calculer le pourcentage de valeurs manquantes pour chaque variable
missing_percentages = (df.isnull().sum() / len(df)) * 100

# Créer un DataFrame contenant les pourcentages de valeurs manquantes
missing_data = pd.DataFrame({'MissingPercentage': missing_percentages})
missing_data 

In [ ]:
# Filtrer les variables où le pourcentage de valeurs manquantes est supérieur à 80%
variables_with_high_missing = missing_data[missing_data['MissingPercentage'] > 80]

# Afficher les noms des variables avec un pourcentage élevé de valeurs manquantes
print("Variables avec plus de 80% de valeurs manquantes:")
print(variables_with_high_missing.index.tolist())

In [ ]:
# On supprime les colonnes avec plus de 80% de valeurs vides 
df = df.drop(["Enedc (g/km)", "Ernedc (g/km)"], axis = 1)

### <font color='blue'> Renommer les variables :  </font>

In [ ]:
rename = {"Country" : "Pays",
          "m (kg)" : "Masse",
          "Ewltp (g/km)" : "Emission_co2_(g/km)",
          "W (mm)" : "Empattement",
          "At1 (mm)" : "Steering_axle_width",
          "At2 (mm)" : "Other_axle_width",
          "Ft" : "Fuel_type",
          "ec (cm3)" : "Cylindrée",
          "ep (KW)" : "Puissance_moteur",
          "z (Wh/km)" : "Consommation_électrique",
          "Erwltp (g/km)" : "Réduction_émissions_CO2(g/km)",
          "Electric range (km)" : "Autonomie_électrique",
          "Mp" : "Fabricant"}

df = df.rename(rename, axis = 1)

### <font color='blue'> Traiter les modalités sous-représentés et dont on n'a pas besoins: </font>

In [ ]:
#Calculer les pourcentages de chaque modalité de la variable Fuel_type : 
modalite_counts = df['Fuel_type'].value_counts(normalize=True)
print(modalite_counts)
# On supprime les modalités sous repésentées dans la la variable Fuel_type  
df = df.loc[(df["Fuel_type"]!="unknown")&(df["Fuel_type"]!="hydrogen")&(df["Fuel_type"]!="ng")&(df["Fuel_type"]!="e85")&(df["Fuel_type"]!="diesel/electric")]


In [ ]:
# On supprime les modalités sous repésentées dans la la variable Pays
print(df["Pays"].value_counts(normalize=True))
df = df.loc[(df["Pays"]=="DE")|(df["Pays"]=="FR")|(df["Pays"]=="IT")|(df["Pays"]=="ES")]


In [ ]:
# Suppression de la modalité "electric" de Fuel_type
df = df.loc[df["Fuel_type"]!="electric"]


### <font color='blue'> Gérer les Nan des autres variables: </font>

In [ ]:
# Nettoyage des variables spécifiques (non valable pour tous les types de véhicules)
# il faut remplacer les NAN par des 0 dans les colonnes Autonomie_électrique et Consommation_électrique pour les véhicules non hybrides
# Remplissage de NAN par 0 pour la colonne Réduction_émissions_CO2
df.loc[df["Fuel_type"] != "petrol/electric","Autonomie_électrique"] = 0
df.loc[df["Fuel_type"] != "petrol/electric","Consommation_électrique"] = 0
df["Réduction_émissions_CO2(g/km)"] = df["Réduction_émissions_CO2(g/km)"].fillna(0)


### <font color='blue'> Supprimer les valeurs manquantes que nous ne pouvons pas remplacer: </font>

In [ ]:
df = df.dropna()

### <font color='blue'> features de variables: </font>

In [ ]:
# Ajout de Moyenne_largeur_essieu et suppression de At1 et At2
df["Moyenne_largeur_essieu"]= (df["Steering_axle_width"]+df["Other_axle_width"])/2
df = df.drop(["Steering_axle_width", "Other_axle_width"], axis = 1)

### <font color='blue'> Gérer les outliers: </font>

In [ ]:
df = df.loc[(df["Masse"]>400)&(df["Masse"]<2160)]
df = df.loc[(df["Puissance_moteur"]>20)&(df["Puissance_moteur"]<200)]


### <font color='blue'> Ajout de la Feature 1: </font>

In [ ]:
#Conversion de l'empattement de millimètre en mètre
df["Empattement"] = df["Empattement"]/1000
#Conversion de la moyenne_largeur_essieu de millimètre en mètre
df["Moyenne_largeur_essieu"] = df["Moyenne_largeur_essieu"]/1000

# Ajout de la Feature 1 
df['feature1']=df['Masse']/(df['Empattement']*(df['Moyenne_largeur_essieu']))
df = df.drop(["Masse", "Empattement", "Moyenne_largeur_essieu"], axis = 1)


In [ ]:
df.shape

### <font color='blue'> Heatmap des corrélations: </font>

In [ ]:
# Sélectionner uniquement les colonnes numériques
numeric_df = df.select_dtypes(include=['number'])

# Calculer la matrice de corrélation
correlation_matrix = numeric_df.corr()

# Créer la heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Heatmap des variables quantitatives du dataset') 
plt.show()


# <font color='red'>3/ Data visualisation :</font>

In [ ]:
plt.figure(figsize=(10, 8))

# Calcul de la moyenne des émissions de CO2 par marque
avg_co2_by_brand = df.groupby('Fabricant')['Emission_co2_(g/km)'].mean().sort_values()

# Création du graphique à barres horizontales
bars = avg_co2_by_brand.plot(kind='barh', color='skyblue')  # Barres horizontales avec couleur skyblue
plt.xlabel('Émissions moyennes de CO2')
plt.ylabel('Fabricants')
plt.title('Émissions moyennes de CO2 des constructeurs automobiles sans voitures electriques')
# Modification de la taille de la police des étiquettes de l'axe des catégories
plt.yticks(fontsize=8)

# Ajout des annotations (valeurs) sur chaque barre
for index, value in enumerate(avg_co2_by_brand):
    # Affichage de la valeur de l'émission de CO2 au-dessus de chaque barre
    plt.text(value, index, f'{value:.1f} g/km', va='center', ha='left', fontsize=8, color='black')
    
# Sauvegarde du graphique en tant que fichier PNG
plt.savefig('emissions_co2_par_marque1.png', dpi=300)  # Spécifiez le nom du fichier et la résolution (dpi)

plt.show()



# <font color='red'>Dichotomiser les variables categorielles :</font>

In [ ]:
# Afficher les noms des colonnes catégorielles
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Utilisation de la fonction get_dummies de Pandas pour la dichotomisation
fuel_dummies = pd.get_dummies(df['Fuel_type'], prefix='Fuel', drop_first=True)
df = pd.concat([df, fuel_dummies], axis=1)

country_dummies = pd.get_dummies(df['Pays'], prefix='Country', drop_first=True)
df = pd.concat([df, country_dummies], axis=1)

manufacturer_dummies = pd.get_dummies(df['Fabricant'], prefix='Manufacturer', drop_first=True)
df = pd.concat([df, manufacturer_dummies], axis=1)
#supprimer les variables 'pays', 'fabriquant' et 'fuel_type' :
df = df.drop(['Fuel_type','Pays', 'Fabricant'], axis=1)  

Au cours de la modélisation nous allons analyser 3 dataframes : 
- df qui est le dataset complet comportanat les voitures thermiques et hybrides
- df_thermiques pour les voitures thermiques 
- df_hybrides pour les voitures hybries


### <font color='blue'> Datasets voitures thermiques et hybrides: </font>

In [ ]:
df_hybride = df[df['Consommation_électrique']!=0.0]
df_hybride = df_hybride.drop(columns=['Fuel_lpg', 'Fuel_petrol'])

df_thermique = df[df['Consommation_électrique']==0.0]
df_thermique = df_thermique.drop(columns=['Autonomie_électrique','Consommation_électrique','Fuel_petrol/electric'])


### <font color='blue'> Heatmap numero 2: </font>

In [ ]:
# Calculer la matrice de corrélation
correlation_matrix = df.corr()

# Créer la heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Heatmap des corrélations')
plt.show()


### <font color='blue'> Supprimer Consommation electrique et Fuel_petrol/electric : </font>
D'après la heatmap on voit une correlation entre 'Consommation electrique', 'Fuel_type/electric' et 'Autonomie electriques'. Nous avons choisi de ne garder que la variable 'Autonomie electriques' pour representer les voitures electriques.

In [ ]:
df = df.drop(['Consommation_électrique','Fuel_petrol/electric'], axis = 1)
df_hybride = df_hybride.drop('Consommation_électrique', axis=1)


### <font color='blue'> Heatmap numero 3: </font>


In [ ]:
# Calculer la matrice de corrélation
correlation_matrix = df.corr()

# Créer la heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Heatmap des corrélations')
plt.show()


### <font color='blue'> Exporter les Datasets au format CSV: </font>

In [ ]:
df_cleaned = df.to_csv('data_cleaned.csv')
df_thermique = df_thermique.to_csv('data_thermique.csv')
df_hybride = df_hybride.to_csv('data_hybride.csv')
